In [1]:
import datetime
from datetime import date
from datetime import timedelta
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import os
import sys
import xarray as xr
import pandas as pd
import netCDF4 as nc
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from scipy.integrate import trapz
from scipy.stats import norm
from scipy.interpolate import interp1d
from shapely.geometry import Point
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [69]:
gap_filled_SWE = 'C:/Users/walimunige.rupasingh/OneDrive - University of Calgary/Documents/Reasearch/Snow_drought_indicator/snowdroughtindex/data/input_data/Case_study/Test/SWE_1979_2022_gapfilled_basin05BB001_Bow.nc'
Bow_at_Banff = 'C:/Users/walimunige.rupasingh/OneDrive - University of Calgary/Documents/Reasearch/Snow_drought_indicator/snowdroughtindex/data/input_data/Bow_at_Banff/basins_testcases.shp'
output_path = 'C:/Users/walimunige.rupasingh/OneDrive - University of Calgary/Documents/Reasearch/Snow_drought_indicator/snowdroughtindex/data/output_data/Case_study/classification/'
P_path = 'C:/Users/walimunige.rupasingh/OneDrive - University of Calgary/Documents/Reasearch/Snow_drought_indicator/snowdroughtindex/data/input_data/Case_study/Test/SCDNA_v1.1_testcases.nc'
RDRS = 'C:/Users/walimunige.rupasingh/OneDrive - University of Calgary/Documents/Reasearch/Snow_drought_indicator/snowdroughtindex/data/input_data/Case_study/adjusted_merged_daily_precipitation.nc'

In [3]:
SWE = xr.open_dataset(gap_filled_SWE)

SWE_testbasin = SWE.to_dataframe()

display(SWE_testbasin)

SWE  flag donor_stations        lat         lon  \
time       station_id                                                           
1979-01-01 ALE-05BA811        NaN   NaN                 51.700001 -116.466667   
           SCD-AL003          NaN   NaN                 51.183334 -115.566666   
           ALE-05BA801        NaN   NaN                 51.416668 -116.183334   
           ALE-05BA802        NaN   NaN                 51.433334 -116.166664   
           ALE-05BA806        NaN   NaN                 51.416668 -116.233330   
...                           ...   ...            ...        ...         ...   
2022-07-31 ALE-05BA814        NaN   0.0                 51.683334 -116.383331   
           ALE-05BB803        NaN   0.0                 51.083332 -115.783333   
           ALE-05CA805        NaN   0.0                 51.533333 -116.050003   
           ALE-05BB803P  40.11483   0.0                 51.083332 -115.783333   
           ALE-05CA805P   3.89600   0.0                 51.540554 -116.056389   

                                    station_name  
time       station_id                             
1979-01-01 ALE-05BA811          BOW SUMMIT (OLD)  
           SCD-AL003                       BANFF  
           ALE-05BA801                 BOW RIVER  
           ALE-05BA802           PIPESTONE UPPER  
           ALE-05BA806               MIRROR LAKE  
...                                          ...  
2022-07-31 ALE-05BA814            KATHERINE LAKE  
           ALE-05BB803          SUNSHINE VILLAGE  
           ALE-05CA805            SKOKI MOUNTAIN  
           ALE-05BB803P  SUNSHINE VILLAGE PILLOW  
           ALE-05CA805P       SKOKI LODGE PILLOW  

[222852 rows x 6 columns]

In [4]:
#Take mean SWE for each day for whole basin
daily_mean_SWE = SWE_testbasin.groupby('time')['SWE'].mean().reset_index()

# Rename the columns for clarity
daily_mean_SWE.columns = ['Date', 'mean_SWE']

# from 1980
daily_mean_SWE = daily_mean_SWE[daily_mean_SWE['Date'].dt.year >= 1980]

# Display the result
print(daily_mean_SWE)

            Date    mean_SWE
365   1980-01-01  226.380000
366   1980-01-02  250.726429
367   1980-01-03  231.135357
368   1980-01-04  232.386786
369   1980-01-05  233.548214
...          ...         ...
15913 2022-07-27   21.446791
15914 2022-07-28   21.756103
15915 2022-07-29   22.024836
15916 2022-07-30   21.994835
15917 2022-07-31   22.005415

[15553 rows x 2 columns]


In [5]:
# get peak SWE for each year 1991-2020
peak_SWE = daily_mean_SWE[daily_mean_SWE['Date'].dt.year.isin(range(1991, 2020))].groupby(
    daily_mean_SWE['Date'].dt.year)['mean_SWE'].max().reset_index()

# Rename the columns for clarity
peak_SWE.columns = ['Year', 'peak_SWE']

# Display the result
display(peak_SWE)

#get average peak SWE for 1991-2020
average_peak_SWE = peak_SWE['peak_SWE'].mean()
print(average_peak_SWE)

# save the peak SWE data
peak_SWE.to_csv(output_path + 'peak_SWE.csv', index=False)


,Year,peak_SWE
0,1991,693.401337
1,1992,457.347583
2,1993,362.014795
3,1994,548.172378
4,1995,596.887736
5,1996,671.307925
6,1997,569.432354
7,1998,393.666946
8,1999,580.584284
9,2000,585.950000


519.5685475877784


In [6]:
# Onset of the season. 5% of the average_peak_SWE
onset_SWE = average_peak_SWE * 0.05
print(onset_SWE)

25.978427379388922


In [ ]:
# select complete winter season nov 1st to May 1st next year and add a season year column
winter_season = daily_mean_SWE[(daily_mean_SWE['Date'].dt.month >= 11) | (daily_mean_SWE['Date'].dt.month <= 5)]
winter_season['Season_year'] = winter_season['Date'].dt.year
winter_season['Season_year'] = winter_season['Season_year'].where(winter_season['Date'].dt.month >= 10, winter_season['Season_year'] - 1)
# drop 1979
winter_season = winter_season[winter_season['Season_year'] >= 1980]

display(winter_season)


C:\Users\walimunige.rupasingh\AppData\Local\Temp\ipykernel_1580\855835743.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winter_season['Season_year'] = winter_season['Date'].dt.year
C:\Users\walimunige.rupasingh\AppData\Local\Temp\ipykernel_1580\855835743.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winter_season['Season_year'] = winter_season['Season_year'].where(winter_season['Date'].dt.month >= 10, winter_season['Season_year'] - 1)


,Date,mean_SWE,Season_year
670,1980-11-01,23.605000,1980
671,1980-11-02,23.930000,1980
672,1980-11-03,24.100000,1980
673,1980-11-04,24.195000,1980
674,1980-11-05,24.515000,1980
...,...,...,...
15852,2022-05-27,552.421699,2021
15853,2022-05-28,566.509663,2021
15854,2022-05-29,564.152801,2021
15855,2022-05-30,560.143669,2021


In [8]:
# Initialize an empty DataFrame to store the results
onset_to_peak_SWE = pd.DataFrame()

# Iterate through each season year
for year in winter_season['Season_year'].unique():
    # Filter the data for the current season year
    season_data = winter_season[winter_season['Season_year'] == year]
    
    # Find the onset date
    onset_date = season_data[season_data['mean_SWE'] >= onset_SWE]['Date'].min()
    
    # Find the peak date
    peak_date = season_data[season_data['mean_SWE'] == season_data['mean_SWE'].max()]['Date'].values[0]
    
    # Select data from onset to peak
    selected_data = season_data[(season_data['Date'] >= onset_date) & (season_data['Date'] <= peak_date)]
    
    # Append the selected data to the result DataFrame
    onset_to_peak_SWE = pd.concat([onset_to_peak_SWE, selected_data])

# Display the result
display(onset_to_peak_SWE)

,Date,mean_SWE,Season_year
675,1980-11-06,27.750000,1980
676,1980-11-07,32.155000,1980
677,1980-11-08,39.045000,1980
678,1980-11-09,30.843333,1980
679,1980-11-10,33.706667,1980
...,...,...,...
15815,2022-04-20,612.574682,2021
15816,2022-04-21,584.810744,2021
15817,2022-04-22,594.028859,2021
15818,2022-04-23,595.556964,2021


In [9]:
# save the onset to peak SWE data
onset_to_peak_SWE.to_csv(output_path + 'onset_to_peak_SWE.csv', index=False)

In [10]:
# open the shapefile
basin = gpd.read_file(Bow_at_Banff)

#select 05BB001 basin
basin = basin[basin['Station_ID'] == '05BB001']

# Display the basin shapefile
display(basin)

,Station_ID,Station_Na,Area_km2,Source,geometry
0,05BB001,BOW RIVER AT BANFF,2210.56,WSC_HYDAT,"POLYGON ((-115.83512 51.07677, -115.83524 51.0..."


In [11]:
met_stations_ds = xr.open_dataset(P_path)

#met_stations_ds = P.to_dataframe()

display(met_stations_ds)

<xarray.Dataset>
Dimensions:    (nchar: 13, nstn: 124, lle: 3, nday: 14610, nsf: 3)
Dimensions without coordinates: nchar, nstn, lle, nday, nsf
Data variables: (12/13)
    ID         (nchar, nstn) |S1 ...
    LLE        (lle, nstn) float32 ...
    date       (nday) int32 ...
    sflag      (nsf, nstn) |S1 ...
    prcp       (nstn, nday) float32 ...
    prcp_flag  (nstn, nday) float64 ...
    ...         ...
    tmin       (nstn, nday) float32 ...
    tmin_flag  (nstn, nday) float64 ...
    tmin_kge   (nstn) float32 ...
    tmax       (nstn, nday) float32 ...
    tmax_flag  (nstn, nday) float64 ...
    tmax_kge   (nstn) float32 ...
Attributes:
    info:     Sample data created by Louise Arnal (USask) on August 17, 2023,...

In [12]:

# Convert meteorological stations DataSet to GeoDataFrame for further analysis
data = {'lon': met_stations_ds.LLE.data[1], 
        'lat': met_stations_ds.LLE.data[0],
        'elev': met_stations_ds.LLE.data[2]} 
df = pd.DataFrame(data)
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
met_stations_gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

display(met_stations_gdf)


,lon,lat,elev,geometry
0,-116.050003,50.880001,1170.400024,POINT (-116.05000 50.88000)
1,-116.059998,50.630001,935.000000,POINT (-116.06000 50.63000)
2,-116.930000,50.950001,1370.000000,POINT (-116.93000 50.95000)
3,-116.260002,50.820000,823.000000,POINT (-116.26000 50.82000)
4,-116.709999,50.750000,1529.000000,POINT (-116.71000 50.75000)
...,...,...,...,...
119,-106.919998,38.599998,2380.000000,POINT (-106.92000 38.60000)
120,-107.800003,39.529999,1622.000000,POINT (-107.80000 39.53000)
121,-107.719719,39.527779,1683.099976,POINT (-107.71972 39.52778)
122,-107.733002,39.533001,1690.000000,POINT (-107.73300 39.53300)


In [92]:
sys.path.append('C:/Users/walimunige.rupasingh/OneDrive - University of Calgary/Documents/Reasearch/Snow_drought_indicator/snowdroughtindex/notebooks/') 
from functions import extract_stations_in_basin

In [93]:
test_basin_id = '05BB001'
# Extract meteorological stations within test basin (+ optional buffer) and save info to DataFrame
met_stations_in_basin = extract_stations_in_basin(met_stations_gdf, basin, test_basin_id, buffer_km=0)[0]

display(met_stations_in_basin)

print('There are',str(len(met_stations_in_basin.index)),'meteorological stations within the test basin.')

,lon,lat,elev,geometry,basin
13,-115.550003,51.189999,1396.900024,POINT (-115.55000 51.19000),05BB001
16,-115.919998,51.270000,1524.000000,POINT (-115.92000 51.27000),05BB001
27,-116.220001,51.430000,1524.000000,POINT (-116.22000 51.43000),05BB001
31,-115.779999,51.070000,2186.899902,POINT (-115.78000 51.07000),05BB001
34,-115.566704,51.183300,1384.000000,POINT (-115.56670 51.18330),05BB001
42,-115.550003,51.200001,1396.000000,POINT (-115.55000 51.20000),05BB001


There are 6 meteorological stations within the test basin.


In [94]:
# Extract only precipitation observations in the test basin
P_testbasin_da = met_stations_ds.sel(nstn=met_stations_in_basin.index).prcp.dropna(dim='nstn', how='all')

display(P_testbasin_da)

<xarray.DataArray 'prcp' (nstn: 5, nday: 14610)>
array([[ 0.      ,  0.363043,  0.      , ...,  7.1     ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      , ..., 20.466087,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      , ..., 14.470592,  0.62419 ,  0.53611 ],
       [ 0.      ,  0.      ,  0.      , ..., 28.7     ,  0.3     ,  0.3     ],
       [ 0.      ,  0.      ,  0.      , ..., 28.702   ,  0.254   ,  0.254   ]],
      dtype=float32)
Dimensions without coordinates: nstn, nday
Attributes:
    description:  SCD precipitation (mm/day)

In [95]:
# Convert the test basin precipitation DataArray to a Pandas DataFrame for further analysis
P_testbasin_df = P_testbasin_da.to_dataframe().unstack().T
P_testbasin_df['date'] = pd.to_datetime(met_stations_ds.sel(nstn=met_stations_in_basin.index).date.data, format='%Y%m%d')
P_testbasin_df = P_testbasin_df.set_index('date')

# Rename the stations with a straightforward index for the infilling steps
P_testbasin_df.columns = ['precip_' + str(col+1) for col in P_testbasin_df.columns]

display(P_testbasin_df)

,precip_1,precip_2,precip_3,precip_4,precip_5
date,,,,,
1979-01-01,0.000000,0.000000,0.000000,0.000000,0.000
1979-01-02,0.363043,0.000000,0.000000,0.000000,0.000
1979-01-03,0.000000,0.000000,0.000000,0.000000,0.000
1979-01-04,0.000000,0.000000,0.000000,0.000000,0.000
1979-01-05,0.000000,0.000000,0.000000,0.000000,0.000
...,...,...,...,...,...
2018-12-27,0.000000,0.000000,0.000000,0.000000,0.000
2018-12-28,21.799999,0.000000,0.000000,0.000000,0.000
2018-12-29,7.100000,20.466087,14.470592,28.700001,28.702


In [96]:
# Calculate the mean precipitation for the test basin
P_testbasin_df['mean_P'] = P_testbasin_df.mean(axis=1)  # axis=1 calculates the mean across columns

# drop 1979 and keep only mean_P
P_testbasin_df = P_testbasin_df[P_testbasin_df.index.year >= 1980][['mean_P']]

display(P_testbasin_df)

,mean_P
date,
1980-01-01,0.000000
1980-01-02,1.250589
1980-01-03,0.080000
1980-01-04,0.800000
1980-01-05,0.280000
...,...
2018-12-27,0.000000
2018-12-28,4.360000
2018-12-29,19.887735


In [98]:
# define water year
P_testbasin_df['water_year'] = P_testbasin_df.index.year
P_testbasin_df['water_year'] = P_testbasin_df['water_year'].where(P_testbasin_df.index.month >= 10, P_testbasin_df['water_year'] - 1)

#correct mean P for 40% underestimation
P_testbasin_df['mean_P'] = P_testbasin_df['mean_P'] 

# drop 2018
P_testbasin_df = P_testbasin_df[P_testbasin_df['water_year'] != 2018]

#drop 1979
P_testbasin_df = P_testbasin_df[P_testbasin_df['water_year'] >= 1980]


display(P_testbasin_df)

,mean_P,water_year
date,,
1980-10-01,0.000000,1980
1980-10-02,0.000000,1980
1980-10-03,0.000000,1980
1980-10-04,0.000000,1980
1980-10-05,0.000000,1980
...,...,...
2018-09-26,0.000000,2017
2018-09-27,2.121871,2017
2018-09-28,0.714000,2017


In [99]:
# Calculate the cumulative precipitation for each water year
P_testbasin_df['cumulative_P'] = P_testbasin_df.groupby('water_year')['mean_P'].cumsum()

display(P_testbasin_df)

,mean_P,water_year,cumulative_P
date,,,
1980-10-01,0.000000,1980,0.000000
1980-10-02,0.000000,1980,0.000000
1980-10-03,0.000000,1980,0.000000
1980-10-04,0.000000,1980,0.000000
1980-10-05,0.000000,1980,0.000000
...,...,...,...
2018-09-26,0.000000,2017,490.567017
2018-09-27,2.121871,2017,492.688873
2018-09-28,0.714000,2017,493.402893


In [100]:
#get daily precipitation for the winter season
P_testbasin_df['Date'] = pd.to_datetime(P_testbasin_df.index)
winter_season_P = P_testbasin_df[(P_testbasin_df['Date'].dt.month >= 11) | (P_testbasin_df['Date'].dt.month <= 5)]
winter_season_P['Season_year'] = winter_season_P['Date'].dt.year
winter_season_P['Season_year'] = winter_season_P['Season_year'].where(winter_season_P['Date'].dt.month >= 10, winter_season_P['Season_year'] - 1)

display(winter_season_P)

C:\Users\walimunige.rupasingh\AppData\Local\Temp\ipykernel_1580\887664290.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winter_season_P['Season_year'] = winter_season_P['Date'].dt.year
C:\Users\walimunige.rupasingh\AppData\Local\Temp\ipykernel_1580\887664290.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winter_season_P['Season_year'] = winter_season_P['Season_year'].where(winter_season_P['Date'].dt.month >= 10, winter_season_P['Season_year'] - 1)


,mean_P,water_year,cumulative_P,Date,Season_year
date,,,,,
1980-11-01,0.782762,1980,18.783848,1980-11-01,1980
1980-11-02,0.080000,1980,18.863848,1980-11-02,1980
1980-11-03,0.000000,1980,18.863848,1980-11-03,1980
1980-11-04,0.000000,1980,18.863848,1980-11-04,1980
1980-11-05,0.000000,1980,18.863848,1980-11-05,1980
...,...,...,...,...,...
2018-05-27,0.000000,2017,285.293854,2018-05-27,2017
2018-05-28,0.361416,2017,285.655273,2018-05-28,2017
2018-05-29,5.551958,2017,291.207245,2018-05-29,2017


In [101]:
#select common dates from onset to peak SWE and daily mean P
onset_to_peak_SWE = onset_to_peak_SWE.rename(columns={'date': 'Date'})
onset_to_peak_SWE['Date'] = onset_to_peak_SWE['Date'].dt.date
onset_to_peak_SWE['Date'] = pd.to_datetime(onset_to_peak_SWE['Date'], format='%Y-%m-%d')    
onset_to_peak_SWE_P = pd.merge(onset_to_peak_SWE, P_testbasin_df, on='Date', how='inner')

# correct mean P for 40% underestimation
#onset_to_peak_SWE_P['mean_P'] = onset_to_peak_SWE_P['mean_P'] / 0.6

display(onset_to_peak_SWE_P)

,Date,mean_SWE,Season_year,mean_P,water_year,cumulative_P
0,1980-11-06,27.750000,1980,1.628033,1980,20.491880
1,1980-11-07,32.155000,1980,2.147504,1980,22.639385
2,1980-11-08,39.045000,1980,1.107589,1980,23.746973
3,1980-11-09,30.843333,1980,1.501463,1980,25.248436
4,1980-11-10,33.706667,1980,1.758588,1980,27.007025
...,...,...,...,...,...,...
6637,2018-04-16,550.012360,2017,4.077505,2017,250.897934
6638,2018-04-17,557.702805,2017,1.059661,2017,251.957596
6639,2018-04-18,559.794174,2017,0.000000,2017,251.957596
6640,2018-04-19,560.239120,2017,0.000000,2017,251.957596


In [102]:
# save the onset to peak SWE and P data
onset_to_peak_SWE_P.to_csv(output_path + 'onset_to_peak_SWE_P_new.csv', index=False)

In [103]:
# calculate mean_SWE for each season year and select max cumulative_P for each season year
onset_to_peak_SWE_P_mean = onset_to_peak_SWE_P.groupby('Season_year').agg(
    mean_SWE=('mean_SWE', 'mean'),
    mean_cumulative_P=('cumulative_P', 'mean'),
    max_SWE=('mean_SWE', 'max'),
    max_cumulative_P=('cumulative_P', 'max')
).reset_index()

# calculate mean_SWE/max_cumulative_P ratio
onset_to_peak_SWE_P_mean['mean_SWE/mean_cumulative_P'] = onset_to_peak_SWE_P_mean['mean_SWE'] / onset_to_peak_SWE_P_mean['mean_cumulative_P']

display(onset_to_peak_SWE_P_mean)

,Season_year,mean_SWE,mean_cumulative_P,max_SWE,max_cumulative_P,mean_SWE/mean_cumulative_P
0,1980,299.793054,160.132034,811.550000,284.303345,1.872162
1,1981,267.625373,122.765869,833.230000,222.555618,2.179966
2,1982,239.273771,95.296303,599.340000,173.786087,2.510840
3,1983,227.605147,82.364159,462.770844,141.155655,2.763400
4,1984,229.362345,100.751884,439.880000,152.353424,2.276507
5,1985,344.105816,114.656883,680.800034,200.078613,3.001179
6,1986,222.014725,95.591949,468.583524,154.072784,2.322525
7,1987,169.174373,67.760506,455.487950,133.027100,2.496652
8,1988,191.445574,106.580078,514.296462,184.434937,1.796260
9,1989,274.208045,121.095314,587.372620,193.382050,2.264398


In [104]:
# save the onset to peak SWE and P data
onset_to_peak_SWE_P_mean.to_csv(output_path + 'SWE_P_ratio.csv', index=False)

In [106]:
#categorize max_SWE<average_peak_SWE as drought, max_SWE>=average_peak_SWE as No drought
onset_to_peak_SWE_P_mean['Condition'] = np.where(onset_to_peak_SWE_P_mean['max_SWE'] < average_peak_SWE, 'Drought', 'No drought')

#add cumulative_P_anomaly column
onset_to_peak_SWE_P_mean['cumulative_P_anomaly'] = onset_to_peak_SWE_P_mean['mean_cumulative_P'] - onset_to_peak_SWE_P_mean['mean_cumulative_P'].mean()

display(onset_to_peak_SWE_P_mean)

,Season_year,mean_SWE,mean_cumulative_P,max_SWE,max_cumulative_P,mean_SWE/mean_cumulative_P,Condition,cumulative_P_anomaly
0,1980,299.793054,160.132034,811.550000,284.303345,1.872162,No drought,43.633873
1,1981,267.625373,122.765869,833.230000,222.555618,2.179966,No drought,6.267708
2,1982,239.273771,95.296303,599.340000,173.786087,2.510840,No drought,-21.201859
3,1983,227.605147,82.364159,462.770844,141.155655,2.763400,Drought,-34.134003
4,1984,229.362345,100.751884,439.880000,152.353424,2.276507,Drought,-15.746277
5,1985,344.105816,114.656883,680.800034,200.078613,3.001179,No drought,-1.841278
6,1986,222.014725,95.591949,468.583524,154.072784,2.322525,Drought,-20.906212
7,1987,169.174373,67.760506,455.487950,133.027100,2.496652,Drought,-48.737656
8,1988,191.445574,106.580078,514.296462,184.434937,1.796260,Drought,-9.918083
9,1989,274.208045,121.095314,587.372620,193.382050,2.264398,No drought,4.597153


In [107]:
# create two dataframes for drought and no drought
drought = onset_to_peak_SWE_P_mean[onset_to_peak_SWE_P_mean['Condition'] == 'Drought']
no_drought = onset_to_peak_SWE_P_mean[onset_to_peak_SWE_P_mean['Condition'] == 'No drought']

display(drought)
display(no_drought)

,Season_year,mean_SWE,mean_cumulative_P,max_SWE,max_cumulative_P,mean_SWE/mean_cumulative_P,Condition,cumulative_P_anomaly
3,1983,227.605147,82.364159,462.770844,141.155655,2.763400,Drought,-34.134003
4,1984,229.362345,100.751884,439.880000,152.353424,2.276507,Drought,-15.746277
6,1986,222.014725,95.591949,468.583524,154.072784,2.322525,Drought,-20.906212
7,1987,169.174373,67.760506,455.487950,133.027100,2.496652,Drought,-48.737656
8,1988,191.445574,106.580078,514.296462,184.434937,1.796260,Drought,-9.918083
11,1991,221.994574,103.962936,457.347583,147.964035,2.135324,Drought,-12.535225
12,1992,123.507356,74.598183,362.014795,142.577591,1.655635,Drought,-41.899979
17,1997,172.542885,96.054176,393.666946,164.203674,1.796308,Drought,-20.443985
20,2000,185.246334,66.467064,476.440000,140.715347,2.787040,Drought,-50.031097
22,2002,187.581531,61.479218,487.638395,190.828262,3.051137,Drought,-55.018944


,Season_year,mean_SWE,mean_cumulative_P,max_SWE,max_cumulative_P,mean_SWE/mean_cumulative_P,Condition,cumulative_P_anomaly
0,1980,299.793054,160.132034,811.550000,284.303345,1.872162,No drought,43.633873
1,1981,267.625373,122.765869,833.230000,222.555618,2.179966,No drought,6.267708
2,1982,239.273771,95.296303,599.340000,173.786087,2.510840,No drought,-21.201859
5,1985,344.105816,114.656883,680.800034,200.078613,3.001179,No drought,-1.841278
9,1989,274.208045,121.095314,587.372620,193.382050,2.264398,No drought,4.597153
10,1990,402.002867,191.365814,693.401337,260.761658,2.100704,No drought,74.867653
13,1993,218.220227,112.321716,548.172378,190.870148,1.942814,No drought,-4.176445
14,1994,258.001383,123.984558,596.887736,231.601273,2.080915,No drought,7.486397
15,1995,350.846912,191.476685,671.307925,299.350586,1.832322,No drought,74.978523
16,1996,247.299508,134.064850,569.432354,206.974457,1.844626,No drought,17.566689


In [108]:
# standardize cumulative_P_anomaly and mean_SWE/mean_cumulative_P for drought drought dataframe
drought['cumulative_P_anomaly_standardized'] = (drought['cumulative_P_anomaly'] - drought['cumulative_P_anomaly'].mean()) / drought['cumulative_P_anomaly'].std()
drought['mean_SWE/mean_cumulative_P_standardized'] = (drought['mean_SWE/mean_cumulative_P'] - drought['mean_SWE/mean_cumulative_P'].mean()) / drought['mean_SWE/mean_cumulative_P'].std()

display(drought)


C:\Users\walimunige.rupasingh\AppData\Local\Temp\ipykernel_1580\3894695702.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drought['cumulative_P_anomaly_standardized'] = (drought['cumulative_P_anomaly'] - drought['cumulative_P_anomaly'].mean()) / drought['cumulative_P_anomaly'].std()
C:\Users\walimunige.rupasingh\AppData\Local\Temp\ipykernel_1580\3894695702.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drought['mean_SWE/mean_cumulative_P_standardized'] = (drought['mean_SWE/mean_cumulative_P'] - dr

,Season_year,mean_SWE,mean_cumulative_P,max_SWE,max_cumulative_P,mean_SWE/mean_cumulative_P,Condition,cumulative_P_anomaly,cumulative_P_anomaly_standardized,mean_SWE/mean_cumulative_P_standardized
3,1983,227.605147,82.364159,462.770844,141.155655,2.763400,Drought,-34.134003,-0.779224,1.460445
4,1984,229.362345,100.751884,439.880000,152.353424,2.276507,Drought,-15.746277,0.157856,0.377813
6,1986,222.014725,95.591949,468.583524,154.072784,2.322525,Drought,-20.906212,-0.105106,0.480138
7,1987,169.174373,67.760506,455.487950,133.027100,2.496652,Drought,-48.737656,-1.523459,0.867316
8,1988,191.445574,106.580078,514.296462,184.434937,1.796260,Drought,-9.918083,0.454874,-0.690038
11,1991,221.994574,103.962936,457.347583,147.964035,2.135324,Drought,-12.535225,0.321498,0.063887
12,1992,123.507356,74.598183,362.014795,142.577591,1.655635,Drought,-41.899979,-1.174995,-1.002725
17,1997,172.542885,96.054176,393.666946,164.203674,1.796308,Drought,-20.443985,-0.081550,-0.689932
20,2000,185.246334,66.467064,476.440000,140.715347,2.787040,Drought,-50.031097,-1.589375,1.513008
22,2002,187.581531,61.479218,487.638395,190.828262,3.051137,Drought,-55.018944,-1.843567,2.100242


In [109]:
# k-means clustering
from sklearn.cluster import KMeans
cluster_feature = drought[['mean_SWE/mean_cumulative_P_standardized','cumulative_P_anomaly_standardized']]

#K = 3
kmean = KMeans(n_clusters=3,random_state=0)

#Fit the model and predict cluster labels

drought['cluster'] = kmean.fit_predict(cluster_feature)

# Print the cluster centers and assigned clusters
print("Cluster centers:\n", kmean.cluster_centers_)
print("\nAssigned clusters:\n", drought[['Season_year', 'cluster']])

# Assign names to clusters based on their characteristics
cluster_labels = {
    0: 'Dry',        # Replace 0, 1, 2 with the actual cluster numbers after inspection
    1: 'Warm',
    2: 'Warm & Dry'
    
}

# Map cluster numbers to names in the DataFrame
drought['cluster_name'] = drought['cluster'].map(cluster_labels)

plt.figure(figsize=(10, 6))

# Define colors for each cluster (adjust colors as needed)
cluster_colors = {'Dry': 'blue','Warm': 'red',  'Warm & Dry': 'purple'}

plt.scatter(drought['cumulative_P_anomaly'], drought['mean_SWE/mean_cumulative_P'], c=drought['cluster_name'].map(cluster_colors))
plt.scatter(no_drought['cumulative_P_anomaly'], no_drought['mean_SWE/mean_cumulative_P'], c='grey', label='No drought')

plt.xlabel('Cumulative Precipitation Anomaly (mm)')
plt.ylabel('SWE/P Ratio')
# Create a legend for named clusters
for name, color in cluster_colors.items():
    plt.bar(0, 0, color=color, label=name)
plt.legend(title='Cluster')
plt.title('K-means Clustering of Seasonal Data (K=3)')
plt.ylim(0)
plt.show()
plt.savefig(output_path + 'K-means_Clustering.png')


c:\Users\walimunige.rupasingh\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Cluster centers:
 [[-0.87453731  0.95290009]
 [ 1.48525285 -1.43390618]
 [ 0.02259372 -0.11683456]]

Assigned clusters:
     Season_year  cluster
3          1983        1
4          1984        2
6          1986        2
7          1987        1
8          1988        0
11         1991        2
12         1992        2
17         1997        2
20         2000        1
22         2002        1
23         2003        0
24         2004        0
25         2005        2
27         2007        2
28         2008        0
29         2009        2
32         2012        0
34         2014        0
35         2015        0


C:\Users\walimunige.rupasingh\AppData\Local\Temp\ipykernel_1580\668563759.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drought['cluster'] = kmean.fit_predict(cluster_feature)
C:\Users\walimunige.rupasingh\AppData\Local\Temp\ipykernel_1580\668563759.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drought['cluster_name'] = drought['cluster'].map(cluster_labels)
C:\Users\walimunige.rupasingh\AppData\Local\Temp\ipykernel_1580\668563759.py:43: UserWarning: Matplotlib is currently using agg, which is

# RDRS

In [140]:
# open the RDRS dataset
RDRS_ds = xr.open_dataset(RDRS)

# convert to dataframe
RDRS_ds = RDRS_ds.to_dataframe()

display(RDRS_ds)

,coordinate_id,lon,lat,daily_precipitation
time,,,,
1980-01-01,1.0,-115.625092,50.972343,0.254825
1980-01-01,2.0,-115.673279,51.057087,0.236767
1980-01-01,3.0,-115.856125,51.111439,0.303147
1980-01-01,4.0,-115.721619,51.141815,0.208841
1980-01-01,5.0,-115.586929,51.172028,0.096979
...,...,...,...,...
2024-10-29,56.0,-116.076141,51.573727,0.148780
2024-10-29,58.0,-116.398376,51.596443,0.148697
2024-10-29,60.0,-116.262238,51.627586,0.159498


In [141]:
#convert to geodataframe
geometry = [Point(xy) for xy in zip(RDRS_ds['lon'], RDRS_ds['lat'])]
RDRS_gdf = gpd.GeoDataFrame(RDRS_ds, crs="EPSG:4326", geometry=geometry)    

display(RDRS_gdf)

,coordinate_id,lon,lat,daily_precipitation,geometry
time,,,,,
1980-01-01,1.0,-115.625092,50.972343,0.254825,POINT (-115.62509 50.97234)
1980-01-01,2.0,-115.673279,51.057087,0.236767,POINT (-115.67328 51.05709)
1980-01-01,3.0,-115.856125,51.111439,0.303147,POINT (-115.85612 51.11144)
1980-01-01,4.0,-115.721619,51.141815,0.208841,POINT (-115.72162 51.14182)
1980-01-01,5.0,-115.586929,51.172028,0.096979,POINT (-115.58693 51.17203)
...,...,...,...,...,...
2024-10-29,56.0,-116.076141,51.573727,0.148780,POINT (-116.07614 51.57373)
2024-10-29,58.0,-116.398376,51.596443,0.148697,POINT (-116.39838 51.59644)
2024-10-29,60.0,-116.262238,51.627586,0.159498,POINT (-116.26224 51.62759)


In [142]:
# Perform spatial join to extract data for the test basin
test_basin_data = gpd.sjoin(RDRS_gdf, basin, how="inner", op="intersects")

# Display the extracted data
display(test_basin_data)

,coordinate_id,lon,lat,daily_precipitation,geometry,index_right,Station_ID,Station_Na,Area_km2,Source
time,,,,,,,,,,
1980-01-01,1.0,-115.625092,50.972343,0.254825,POINT (-115.62509 50.97234),0,05BB001,BOW RIVER AT BANFF,2210.56,WSC_HYDAT
1980-01-01,2.0,-115.673279,51.057087,0.236767,POINT (-115.67328 51.05709),0,05BB001,BOW RIVER AT BANFF,2210.56,WSC_HYDAT
1980-01-01,3.0,-115.856125,51.111439,0.303147,POINT (-115.85612 51.11144),0,05BB001,BOW RIVER AT BANFF,2210.56,WSC_HYDAT
1980-01-01,4.0,-115.721619,51.141815,0.208841,POINT (-115.72162 51.14182),0,05BB001,BOW RIVER AT BANFF,2210.56,WSC_HYDAT
1980-01-01,5.0,-115.586929,51.172028,0.096979,POINT (-115.58693 51.17203),0,05BB001,BOW RIVER AT BANFF,2210.56,WSC_HYDAT
...,...,...,...,...,...,...,...,...,...,...
2024-10-29,56.0,-116.076141,51.573727,0.148780,POINT (-116.07614 51.57373),0,05BB001,BOW RIVER AT BANFF,2210.56,WSC_HYDAT
2024-10-29,58.0,-116.398376,51.596443,0.148697,POINT (-116.39838 51.59644),0,05BB001,BOW RIVER AT BANFF,2210.56,WSC_HYDAT
2024-10-29,60.0,-116.262238,51.627586,0.159498,POINT (-116.26224 51.62759),0,05BB001,BOW RIVER AT BANFF,2210.56,WSC_HYDAT


In [165]:
# Ensure 'Date' column is present in the DataFrame
test_basin_data = test_basin_data.reset_index()
if 'time' in test_basin_data.columns:
	test_basin_data = test_basin_data.rename(columns={'time': 'Date'})
test_basin_data['Date'] = pd.to_datetime(test_basin_data['Date'])

# keep only Date, daily_precipitation columns
test_basin_data = test_basin_data[['Date', 'daily_precipitation']]

# daily mean RDRS_P
daily_mean_RDRS_P = test_basin_data.groupby('Date')['daily_precipitation'].mean().reset_index()

display(daily_mean_RDRS_P)

,Date,daily_precipitation
0,1980-01-01,0.174075
1,1980-01-02,0.553970
2,1980-01-03,2.351273
3,1980-01-04,0.784659
4,1980-01-05,0.507873
...,...,...
16369,2024-10-25,0.003105
16370,2024-10-26,0.000000
16371,2024-10-27,1.196224
16372,2024-10-28,1.682913


In [166]:
# define water year
daily_mean_RDRS_P['water_year'] = daily_mean_RDRS_P['Date'].dt.year
daily_mean_RDRS_P['water_year'] = daily_mean_RDRS_P['water_year'].where(daily_mean_RDRS_P['Date'].dt.month >= 10, daily_mean_RDRS_P['water_year'] - 1)

#correct daily precipitation for 40% underestimation
daily_mean_RDRS_P['daily_precipitation'] = daily_mean_RDRS_P['daily_precipitation'] / 0.5

# drop 2018
daily_mean_RDRS_P = daily_mean_RDRS_P[daily_mean_RDRS_P['water_year'] != 2018]

#drop 1979
daily_mean_RDRS_P = daily_mean_RDRS_P[daily_mean_RDRS_P['water_year'] >= 1980]

display(daily_mean_RDRS_P)

,Date,daily_precipitation,water_year
274,1980-10-01,0.560323,1980
275,1980-10-02,0.135040,1980
276,1980-10-03,0.010157,1980
277,1980-10-04,0.000066,1980
278,1980-10-05,0.007397,1980
...,...,...,...
16369,2024-10-25,0.006211,2024
16370,2024-10-26,0.000000,2024
16371,2024-10-27,2.392447,2024
16372,2024-10-28,3.365826,2024


In [167]:
# Calculate the cumulative precipitation for each water year
daily_mean_RDRS_P['cumulative_P'] = daily_mean_RDRS_P.groupby('water_year')['daily_precipitation'].cumsum()

display(daily_mean_RDRS_P)

,Date,daily_precipitation,water_year,cumulative_P
274,1980-10-01,0.560323,1980,0.560323
275,1980-10-02,0.135040,1980,0.695363
276,1980-10-03,0.010157,1980,0.705520
277,1980-10-04,0.000066,1980,0.705586
278,1980-10-05,0.007397,1980,0.712983
...,...,...,...,...
16369,2024-10-25,0.006211,2024,28.129215
16370,2024-10-26,0.000000,2024,28.129215
16371,2024-10-27,2.392447,2024,30.521664
16372,2024-10-28,3.365826,2024,33.887489


In [168]:
# Ensure 'Date' column is present in both dataframes
onset_to_peak_SWE = onset_to_peak_SWE.rename(columns={'date': 'Date'})
onset_to_peak_SWE['Date'] = pd.to_datetime(onset_to_peak_SWE['Date'], format='%Y-%m-%d')
daily_mean_RDRS_P = daily_mean_RDRS_P.rename(columns={'time': 'Date'})
daily_mean_RDRS_P['Date'] = pd.to_datetime(daily_mean_RDRS_P['Date'], format='%Y-%m-%d')

# Select common dates from onset to peak SWE and daily mean P
onset_to_peak_SWE_RDRS = pd.merge(onset_to_peak_SWE, daily_mean_RDRS_P, on='Date', how='inner')


display(onset_to_peak_SWE_RDRS)

,Date,mean_SWE,Season_year,daily_precipitation,water_year,cumulative_P
0,1980-11-06,27.750000,1980,1.788661,1980,39.278515
1,1980-11-07,32.155000,1980,4.686693,1980,43.965206
2,1980-11-08,39.045000,1980,1.254156,1980,45.219360
3,1980-11-09,30.843333,1980,4.068890,1980,49.288250
4,1980-11-10,33.706667,1980,5.548077,1980,54.836327
...,...,...,...,...,...,...
7143,2022-04-20,612.574682,2021,0.177935,2021,176.805664
7144,2022-04-21,584.810744,2021,3.076361,2021,179.882034
7145,2022-04-22,594.028859,2021,1.286734,2021,181.168762
7146,2022-04-23,595.556964,2021,0.252983,2021,181.421753


In [169]:
# calculate mean_SWE for each season year and select max cumulative_P for each season year
onset_to_peak_SWE_RDRS_mean = onset_to_peak_SWE_RDRS.groupby('Season_year').agg(
    mean_SWE=('mean_SWE', 'mean'),
    mean_cumulative_P=('cumulative_P', 'mean'),
    max_SWE=('mean_SWE', 'max'),
    max_cumulative_P=('cumulative_P', 'max')
).reset_index()

# calculate mean_SWE/max_cumulative_P ratio
onset_to_peak_SWE_RDRS_mean['mean_SWE/mean_cumulative_P'] = onset_to_peak_SWE_RDRS_mean['mean_SWE'] / onset_to_peak_SWE_RDRS_mean['mean_cumulative_P']

# keep 1980 to 2017
onset_to_peak_SWE_RDRS_mean = onset_to_peak_SWE_RDRS_mean[onset_to_peak_SWE_RDRS_mean['Season_year'] <= 2017]

display(onset_to_peak_SWE_RDRS_mean)

,Season_year,mean_SWE,mean_cumulative_P,max_SWE,max_cumulative_P,mean_SWE/mean_cumulative_P
0,1980,299.793054,350.400482,811.550000,591.630859,0.855573
1,1981,267.625373,281.162384,833.230000,496.754059,0.951853
2,1982,239.273771,236.943451,599.340000,405.853302,1.009835
3,1983,227.605147,234.912003,462.770844,379.408722,0.968895
4,1984,229.362345,224.935455,439.880000,373.356445,1.019681
5,1985,344.105816,306.051941,680.800034,579.858643,1.124338
6,1986,222.014725,251.337112,468.583524,405.084137,0.883334
7,1987,169.174373,198.687943,455.487950,365.827911,0.851458
8,1988,191.445574,273.139862,514.296462,479.720764,0.700907
9,1989,274.208045,359.248779,587.372620,554.799377,0.763282
